<a href="https://colab.research.google.com/github/Sarth-Patil/PyTorch/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision

In [3]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,                                   
    transform=torchvision.transforms.ToTensor(),   
    download=True,
)

0it [00:00, ?it/s]

9920512it [00:01, 9197993.35it/s]                            


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135275.63it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2035522.96it/s]                           
0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50872.79it/s]            

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
import torch.utils.data as Data
train_loader = Data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

In [25]:
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:1000]/255.
test_y = test_data.test_labels[:1000]

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(        
            nn.Conv2d(
                in_channels=1,             
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential( 
            nn.Conv2d(16, 32, 5, 1, 2),  
            nn.ReLU(), 
            nn.MaxPool2d(2), 
        )
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) 
        output = self.out(x)
        return output, x


In [27]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [0]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.02) 
loss_func = nn.CrossEntropyLoss()                       

In [29]:
for epoch in range(1):
    for step, (x, y) in enumerate(train_loader):
        output = cnn(x)[0]  
        loss = loss_func(output, y) 
        optimizer.zero_grad()     
        loss.backward()     
        optimizer.step() 

        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Step: ', step, ', train loss: %.4f' % loss.data.numpy(), ', test accuracy: %.2f' % (accuracy * 100))

Step:  0 , train loss: 2.3024 , test accuracy: 9.90
Step:  50 , train loss: 0.1141 , test accuracy: 94.00
Step:  100 , train loss: 0.0794 , test accuracy: 92.20
Step:  150 , train loss: 0.2072 , test accuracy: 95.10
Step:  200 , train loss: 0.0844 , test accuracy: 95.00
Step:  250 , train loss: 0.1746 , test accuracy: 94.50
Step:  300 , train loss: 0.0835 , test accuracy: 94.90
Step:  350 , train loss: 0.1801 , test accuracy: 96.30
Step:  400 , train loss: 0.1941 , test accuracy: 96.50
Step:  450 , train loss: 0.1734 , test accuracy: 96.40
Step:  500 , train loss: 0.0981 , test accuracy: 95.40
Step:  550 , train loss: 0.2340 , test accuracy: 96.90
Step:  600 , train loss: 0.0909 , test accuracy: 95.80
Step:  650 , train loss: 0.0182 , test accuracy: 96.70
Step:  700 , train loss: 0.0180 , test accuracy: 96.20
Step:  750 , train loss: 0.3872 , test accuracy: 96.50
Step:  800 , train loss: 0.0405 , test accuracy: 95.50
Step:  850 , train loss: 0.1647 , test accuracy: 97.10
Step:  900 , t

In [32]:
test_output, _ = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print( 'actual numbers',test_y[:10].numpy())
print('predicted numbers',pred_y)


actual numbers [7 2 1 0 4 1 4 9 5 9]
predicted numbers [7 2 1 0 4 1 4 9 5 9]
